1. 데이터 획득
1. 누락값 확인
1. 이상치 확인
1. 데이터 타입 확인, 범주형 > 연속형 변환
1. 입력과 출력 분리
1. 학습/테스트 셋 분리
1. 모델 구현(2진 분류, cross entropy loss) - 학습
1. 테스트셋으로 모델 성능을 평가

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [62]:
df = pd.read_csv('wpbc.txt', header=None)

In [36]:
df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,119513,N,31,18.02,27.6,117.5,1013.0,0.09489,0.1036,0.1086,...,139.7,1436.0,0.1195,0.1926,0.314,0.117,0.2677,0.08113,5.0,5


In [99]:
df[2].min(),df[2].max(),df[2].mean()

(1, 125, 46.73232323232323)

In [37]:
df.isna().sum().sum()

0

In [48]:
(df[34]=='?').mean(),df[34].unique()

(0.020202020202020204,
 array(['5', '2', '0', '?', '10', '1', '20', '6', '13', '4', '17', '15',
        '11', '9', '8', '7', '3', '14', '27', '24', '18', '16', '21'],
       dtype=object))

In [63]:
mask = df[34]=='?'
df.loc[mask, 34]= '0'

In [69]:
df[34] = df[34].astype(int)

In [146]:
df_y = df[1]
df_y2 = df[2]
df_x = df.drop(columns=[0,1,2])
df_x.shape, df_y.shape

((198, 32), (198,))

In [157]:
import numpy as np
data_x = np.array(df_x)
data_y = np.array(df_y)
data_y2 = np.array(df_y2)
data_y[data_y=='R'] = 1
data_y[data_y=='N'] = 0
data_y = data_y.astype(np.int64)
# X 를 [0,1] 로 정규화 ( X - 최소값) / (최대값-최소값) : minMax 정규화
x_min = np.min(data_x, axis=1, keepdims=True)
x_max = np.max(data_x, axis=1, keepdims=True)
data_x = (data_x - x_min)/(x_max-x_min)
np.min(data_x), np.max(data_x), np.mean(data_y),data_y2.shape

(0.0, 1.0, 0.23737373737373738, (198,))

In [153]:
train_x,train_y,train_y2 = data_x[::2] , data_y[::2], data_y2[::2]
test_x, test_y, test_y2  = data_x[1::2], data_y[1::2], data_y2[1::2]

In [154]:
import tensorflow as tf
import tensorflow.contrib.layers as layers

In [161]:
X = tf.placeholder(tf.float32, [None, 32])
Y = tf.placeholder(tf.int64, [None]) # 재발/ 안하냐 (분류)
Y2 = tf.placeholder(tf.float32, [None]) # 언제 재발하느냐 (예측)
Y_f = tf.cast(tf.reshape(Y,[-1,1]), tf.float32)
hidden = layers.fully_connected(X, 100)
hidden1 = layers.fully_connected(hidden, 100)
h = layers.fully_connected(hidden1, 2) # x가 클래스 0일 점수/클래스 1일 점수
h2 = layers.fully_connected(hidden1, 1)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=h))
#재발 하는 환자에 대해서만 언제 재발하느냐를 예측합니다
cost2 = tf.reduce_mean(Y_f * tf.square(h2 - tf.expand_dims(Y2,1)))
#h = tf.maximum(1e-7, h)#0.0000001 보다 작지 않도록 만들기 
#cost = -tf.reduce_mean(1* Y_f*tf.log(h) + (1-Y_f)*tf.log(1-h))
train = tf.train.GradientDescentOptimizer(0.05).minimize(cost + cost2)#joint training
predict = tf.argmax(h, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y, predict),tf.float32)) 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [163]:
for iter in range(9000):
    _, co, co2, acc = sess.run([train, cost,cost2, accuracy], {X:train_x, Y:train_y,Y2:train_y2})
    if iter%1000==0: print(iter, 'cost',co,'acc_cls',acc,'co2',np.sqrt(co2))

0 cost 0.4853674 acc_cls 0.7979798 co2 6.3754663
1000 cost 0.46785933 acc_cls 0.7979798 co2 6.207475
2000 cost 0.47501028 acc_cls 0.7979798 co2 6.3689985
3000 cost 0.47482988 acc_cls 0.7979798 co2 6.4948673
4000 cost 0.47602567 acc_cls 0.7979798 co2 6.1186543
5000 cost 0.47086924 acc_cls 0.7979798 co2 6.1403193
6000 cost 0.47594404 acc_cls 0.7979798 co2 6.0847845
7000 cost 0.47626492 acc_cls 0.7979798 co2 6.17472
8000 cost 0.47055832 acc_cls 0.7979798 co2 6.0729885


https://github.com/sogangori/python_study

In [122]:
acc_test = sess.run(accuracy, {X:test_x, Y:test_y})
print('acc_test',acc_test)

acc_test 0.68686867
